In [1]:
import imaplib
import numpy as np
import pandas as pd
import time
import smtplib
from datetime import date, datetime
from email.message import EmailMessage


In [2]:
EMAIL_ADDRESS = 'sender@gmail.com'
EMAIL_PASSWORD = 'your application password'
RECEIVER_EMAIL_ADDRESS = 'receiver@gmail.com'

In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def sendMail(video, anomaly, analysis):
    SUBJECT = "🚨🚨ANOMALY DETECTED🚨🚨"
    BODY = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>HTML Email</title>
        </head>
        <body>
            <p>Hello User</p>
            <p>An anomaly was detected in your video: {video}.</p>
            <p>Our system has detected that anomaly is <strong>{anomaly}</strong></p>
            </p>Our Analysis: {analysis}</p>
        </body>
        </html>
    """

    # Create message container - the correct MIME type is multipart/alternative.
    msg = MIMEMultipart('alternative')
    msg['Subject'] = SUBJECT
    msg['From'] = EMAIL_ADDRESS
    msg['To'] = RECEIVER_EMAIL_ADDRESS

    # Attach HTML body to the message
    html = MIMEText(BODY, 'html')
    msg.attach(html)

    # Establish a secure SMTP session and log in to the email server
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

        # Send mail
        server.sendmail(EMAIL_ADDRESS, RECEIVER_EMAIL_ADDRESS, msg.as_string())

    print("Email sent successfully!")

In [ ]:
from keras.models import load_model

# Load the model from the HDF5 file
model = load_model('PATH TO MODEL')

In [5]:
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import preprocess_input
from IPython.display import HTML
from base64 import b64encode
# Load the CNN model

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

def predict_video(video_path):
    # Load the video
    video = cv2.VideoCapture(video_path)
    print(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(video.get(cv2.CAP_PROP_FPS))
    
    # Set the desired frame rate and create an empty list for predictions
    frame_rate = 15
    predictions = []
    folder = video_path.split('/')[3]
    if folder == 'Normal_Videos_for_Event_Recognition':
        folder = 'Normal'
    pred_dict = {i: 0 for i in range(0,5)}
        
    labels = {
        0:"Explosion",
        1:"Fighting",
        2:"Normal",
        3:"RoadAccidents",
        4:"Stealing/Snatching"
    }
    prev_max_key = -1
    while True:
        # Read the next frame
        ret, frame = video.read()

        # If the frame couldn't be read, break out of the loop
        if not ret:
            break

        # Get the current frame ID and check if it's a multiple of the desired frame rate
        current_frame_id = video.get(cv2.CAP_PROP_POS_FRAMES)
        check = False
        if current_frame_id % frame_rate == 0:
            # Resize the frame and apply data augmentation
            image = cv2.resize(frame, (224, 224))
            image_data = datagen.apply_transform(image, {
                'shear': 0.2,
                'zoom': 0.2,
                'flip_horizontal': True
            })

            # Add an extra dimension to the image data and preprocess it
            image_data = np.expand_dims(image_data, axis=0)
            image_data = preprocess_input(image_data)

            # Make a prediction and append the predicted class to the list of predictions
            pred_class = model.predict(image_data, verbose=0)
            pred_class = np.argmax(pred_class, axis=1)
            predictions.append(pred_class[0])
            pred_dict[predictions[-1]] += 1
            time = (current_frame_id / frame_rate) / (video.get(cv2.CAP_PROP_FPS) / frame_rate)
            if time >= 5:
                check = True
            
            total = sum(pred_dict.values())

            # calculate the percentage for each value in the dictionary
            result_dict = {k: (v / total) * 100 for k, v in pred_dict.items()}

            if check:
                max_key, max_value = max(result_dict.items(), key=lambda x: x[1])
                if max_key != 2:
                    if max_key != prev_max_key:
                        prev_max_key = max_key
#                         print(labels[max_key], max_value, time)
                    
    video.release()
    results = {}
    for i in result_dict:
        results[labels[i]] = result_dict[i]
    print(results)
    if max_key != 2:
        sendMail(video_path, labels[max_key], results)
    


In [ ]:
predict_video('PATH TO VIDEO')